In [1]:
#importing important libraries

import pandas as pd
import numpy as np

from wordcloud import WordCloud

import tweepy
import re

#libraries for cleaning the words
import nltk

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


#importing important libraries
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

%matplotlib inline

#from googletrans import Translator

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("tweeter api key.csv")


# Creating keys to access tweets from twitter

In [3]:
#Tweeter API credentials
consumer_key=df['Key'][0]
consumer_secret_key=df['Key'][1]
access_token=df['Key'][3]
access_token_secret=df['Key'][4]


In [4]:
#creating the othentication object
authentication=tweepy.OAuthHandler(consumer_key,consumer_secret_key)

#set the access token and access token secret
authentication.set_access_token(access_token,access_token_secret)

#creat the API object while passing the auth information
api=tweepy.API(authentication, wait_on_rate_limit=True)

# Scrapping Tweeter ID from wikipedia

In [5]:

url="https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}
r=requests.get(url, headers=headers)
page=r.content
soup = BeautifulSoup(page,'html.parser')


# Creating a dataframe of clean tweets

In [6]:
#creatring a list of tweeter account
list=[]
for i in tqdm(range(1,51)):
    list.append(soup.findAll(attrs={'class':'plainlinks'})[i].text.split('@')[1])


#extracting the tweets from every user ID
tweets=[]
for i in tqdm(list):
    posts=api.user_timeline(screen_name=i, count=200, tweet_mode="extended")
    for t in tqdm(posts):
        tweets.append(t.full_text)

100%|██████████| 50/50 [00:52<00:00,  1.04s/it]


In [7]:
df_tweets=pd.DataFrame(tweets, columns=['Tweet'])

#showing the first 5 tweet data
df_tweets.head()

,Tweet
0,"Election Day is right around the corner, and I..."
1,"Great news: Thanks to @POTUS, the application ..."
2,Too many entrepreneurs feel like big instituti...
3,I heard there was a group of students from acr...
4,Michelle and I voted early today. Now it’s you...


In [8]:
df_tweets.shape

(9974, 1)

In [9]:
#Creating a function to clean the tweets
def clean(text):
  text=re.sub(r'@[A-Za-z0-9]+','', str(text))#it removes @mentions
  text=re.sub(r'#','', str(text))#removeing '#' symbol from text
  text=re.sub(r':','', str(text))#removeing ':' symbol from text
  text=re.sub(r'RT[\S]+','', str(text))#removing RT(Re Tweet) tagged with the text
  text=re.sub(r'http\S+', '', text)#remove the hyper link
  #we put '?' after s in 'https' because it can be 'http' so '?' will take 's' as 0 or 1 
  #i.e- no matter if 's' condition is fullfilled or not it will remove it
  #r tells the python that it is a raw string
  return text

In [10]:
#Cleaning the text
df_tweets['Tweet']=df_tweets['Tweet'].apply(clean)

#showing cleaned text
df_tweets

,Tweet
0,"Election Day is right around the corner, and I..."
1,"Great news Thanks to , the application for fed..."
2,Too many entrepreneurs feel like big instituti...
3,I heard there was a group of students from acr...
4,Michelle and I voted early today. Now it’s you...
...,...
9969,Happy Birthday. Hope you have a good one. No ...
9970,
9971,LIGHTS UP. Out now.\n\n
9972,


# Creating a list of positive ang negative words

In [11]:
#removing the stopwords and lemmatizing the words
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

def text_prep(x: str) -> list:
    corp = str(x).lower() 
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in stop_words]
    lemmatize = [lemma.lemmatize(w) for w in words]
    return lemmatize

In [12]:
#creating a dataframe of preprocessed words
preprocess_tag = [text_prep(i) for i in df_tweets['Tweet']]
df_tweets["preprocess_txt"] = preprocess_tag

In [13]:
#creating negative and positive score

file = open('negative-words.txt')
neg_words = file.read().split()

file = open('positive-words.txt')
pos_words = file.read().split()

num_pos = df_tweets['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))

df_tweets['POSITIVE SCORE'] = num_pos

num_neg = df_tweets['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))

df_tweets['NEGATIVE SCORE'] = num_neg


In [14]:
df_tweets

,Tweet,preprocess_txt,POSITIVE SCORE,NEGATIVE SCORE
0,"Election Day is right around the corner, and I...","[election, day, right, around, corner, want, c...",0,5
1,"Great news Thanks to , the application for fed...","[great, news, thanks, application, federal, st...",1,3
2,Too many entrepreneurs feel like big instituti...,"[many, entrepreneur, feel, like, big, institut...",1,2
3,I heard there was a group of students from acr...,"[heard, group, student, across, chicago, getti...",0,1
4,Michelle and I voted early today. Now it’s you...,"[michelle, voted, early, today, turn, check, v...",0,0
...,...,...,...,...
9969,Happy Birthday. Hope you have a good one. No ...,"[happy, birthday, hope, good, one, bashing, h]",2,3
9970,,[],0,0
9971,LIGHTS UP. Out now.\n\n,[light],0,0
9972,,[],0,0


In [15]:
sentiment=[]
for i,e in zip(df_tweets['POSITIVE SCORE'],df_tweets['NEGATIVE SCORE']):
    if i-e>0:
        sentiment.append('positive')
    elif i-e<0:
        sentiment.append('negative')
    else:
        sentiment.append('neutral')
df_tweets['sentiment']=sentiment

In [16]:
df_tweets

,Tweet,preprocess_txt,POSITIVE SCORE,NEGATIVE SCORE,sentiment
0,"Election Day is right around the corner, and I...","[election, day, right, around, corner, want, c...",0,5,negative
1,"Great news Thanks to , the application for fed...","[great, news, thanks, application, federal, st...",1,3,negative
2,Too many entrepreneurs feel like big instituti...,"[many, entrepreneur, feel, like, big, institut...",1,2,negative
3,I heard there was a group of students from acr...,"[heard, group, student, across, chicago, getti...",0,1,negative
4,Michelle and I voted early today. Now it’s you...,"[michelle, voted, early, today, turn, check, v...",0,0,neutral
...,...,...,...,...,...
9969,Happy Birthday. Hope you have a good one. No ...,"[happy, birthday, hope, good, one, bashing, h]",2,3,negative
9970,,[],0,0,neutral
9971,LIGHTS UP. Out now.\n\n,[light],0,0,neutral
9972,,[],0,0,neutral


In [17]:
df_tweets['sentiment'].value_counts()

neutral     5240
negative    3567
positive    1167
Name: sentiment, dtype: int64

In [42]:
sentiment_val=[]

for i in df_tweets['sentiment']:
    if i=='positive':
        sentiment_val.append(0)
    elif i=='negative':
        sentiment_val.append(1)
    else:
        sentiment_val.append(2)
df_tweets['sentiment_val']=sentiment_val

In [43]:
df_tweets.head(1)

,Tweet,preprocess_txt,POSITIVE SCORE,NEGATIVE SCORE,sentiment,sentiment_val
0,"Election Day is right around the corner, and I...","[election, day, right, around, corner, want, c...",0,5,negative,1


In [44]:
df_tweets['sentiment_val'].value_counts()

2    5240
1    3567
0    1167
Name: sentiment_val, dtype: int64

In [45]:
for i in df_tweets:
    print(i)

Tweet
preprocess_txt
POSITIVE SCORE
NEGATIVE SCORE
sentiment
sentiment_val


In [46]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(df_tweets['Tweet']).toarray()
y = df_tweets['sentiment_val']

In [47]:
y.value_counts()

2    5240
1    3567
0    1167
Name: sentiment_val, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=0)

In [69]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [70]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
model_3=lr.fit(x_train,y_train)
y_pred=model_3.predict(x_test)

In [71]:
#print('confusion_matrix\n',confusion_matrix(y_test,y_pred))
print('precision_score\n',precision_score(y_test,y_pred,average='weighted'))
print('recall_score\n',recall_score(y_test,y_pred,average='weighted'))
print('f1_score\n',f1_score(y_test,y_pred,average='weighted'))
print('accuracy_score\n',accuracy_score(y_test,y_pred))

precision_score
 0.7794893528006428
recall_score
 0.785084202085004
f1_score
 0.7764599650936119
accuracy_score
 0.785084202085004


In [72]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(class_weight='balanced')
model_1=rf.fit(x_train,y_train)
y_pred=model_1.predict(x_test)

In [73]:
#print('confusion_matrix\n',confusion_matrix(y_test,y_pred))
print('precision_score\n',precision_score(y_test,y_pred,average='weighted'))
print('recall_score\n',recall_score(y_test,y_pred,average='weighted'))
print('f1_score\n',f1_score(y_test,y_pred,average='weighted'))
print('accuracy_score\n',accuracy_score(y_test,y_pred))

precision_score
 0.7934943183367974
recall_score
 0.7919005613472334
f1_score
 0.7896714854797322
accuracy_score
 0.7919005613472334


In [74]:
from xgboost import XGBClassifier 
xgb=XGBClassifier()
model_2=xgb.fit(x_train,y_train)
y_pred=model_2.predict(x_test)

In [75]:
#print('confusion_matrix\n',confusion_matrix(y_test,y_pred))
print('precision_score\n',precision_score(y_test,y_pred,average='weighted'))
print('recall_score\n',recall_score(y_test,y_pred,average='weighted'))
print('f1_score\n',f1_score(y_test,y_pred,average='weighted'))
print('accuracy_score\n',accuracy_score(y_test,y_pred))

precision_score
 0.8033374461914211
recall_score
 0.8011226944667201
f1_score
 0.7871472044308623
accuracy_score
 0.8011226944667201
